In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [3]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import Legend

In [4]:
df = pd.read_csv('NSE-TATAGLOBAL11.csv')
df.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-10-08,208.00,222.25,206.85,216.00,215.15,4642146.0,10062.83
1,2018-10-05,217.00,218.60,205.90,210.25,209.20,3519515.0,7407.06
2,2018-10-04,223.50,227.80,216.15,217.25,218.20,1728786.0,3815.79
3,2018-10-03,230.00,237.50,225.75,226.45,227.60,1708590.0,3960.27
4,2018-10-01,234.55,234.60,221.05,230.30,230.90,1534749.0,3486.05


In [5]:
df2 = df['Close']
df2

0       215.15
1       209.20
2       218.20
3       227.60
4       230.90
         ...  
1230    159.45
1231    160.05
1232    160.15
1233    155.55
1234    155.80
Name: Close, Length: 1235, dtype: float64

In [6]:
scaler = MinMaxScaler(feature_range=(1,2))
df2 = scaler.fit_transform(np.array(df2).reshape(-1,1))

In [7]:
out = []
for i in df2:
    out.append(i[0])
df2= out

In [8]:
train_size = int(len(df2)*0.6)
test_size = len(df2)-train_size
train, test = df2[0:train_size],df2[train_size:len(df2)]
print(len(train), len(test))

741 494


In [9]:
def create_dataset(dataset, look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        a= dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i+look_back])
    return np.array(dataX), np.array(dataY)

In [10]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [11]:
trainX.shape

(730, 10)

In [12]:
trainY.shape

(730,)

In [13]:
testX.shape

(483, 10)

In [14]:
testY.shape

(483,)

In [15]:
trainX = np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX= np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

In [16]:
trainX.shape

(730, 1, 10)

In [17]:
model=Sequential()
model.add(LSTM(4,input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX,trainY,epochs= 100, batch_size=256, verbose=1)

Epoch 1/100


C:\Users\Microsoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.4906
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4062 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3839 
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3368 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2953 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2591 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1943 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1427 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1232 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0570 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0162 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9868 
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9504 
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9183 
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8688 
Epoch 16/100
3/3 ━━━━━━━━━━━━━━━

In [18]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict=scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [19]:
trainScore = math.sqrt(mean_squared_error(trainY[0],trainPredict[:,0]))
print('Train score: %.2f RMSE'%(trainScore))
testScore = math.sqrt(mean_squared_error(testY[0],testPredict[:,0]))
print('Test score: %.2f RMSE'%(testScore))

Train score: 43.61 RMSE
Test score: 20.81 RMSE


In [20]:
trainPredictPlot= np.zeros(len(df2))
trainPredictPlot[:]=np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back]= trainPredict[:,0]

In [21]:
testPredictPlot= np.zeros(len(df2),dtype=np.float64)
testPredictPlot[:]=np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(df2)-1]= testPredict[:,0]

In [22]:
p = figure(width=900,height=300)
p.line(np.arange(len(df['Close'])),df['Close'],legend_label='Actual',color='Black')
tpp=pd.DataFrame(trainPredictPlot,columns=["Close"],index=df.index)['Close']
qpp=pd.DataFrame(testPredictPlot,columns=["Close"],index=df.index)['Close']
p.line(np.arange(len(tpp)),tpp,legend_label='Training',color='blue')
p.line(np.arange(len(qpp)),qpp,legend_label='Testing',color='orange')
p.legend.location="top_right"
show(p)